**Q2.** Try to fit the function $f(x) = \sin(x)$ for $0 < x < 2\pi$ using a **two-layer neural network** with different activation functions:

- ReLU
- GELU
- SiLU
- LeakyReLU
- ELU

Plot the resulting function for each activation function. Also, measure the training and validation loss, where the validation set contains data points from: $-\pi < x < 0$ and $2\pi < x < 3\pi$. Identify which activation function performs best, and provide a justification for the result.

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

In [2]:
learning_rate=0.001
batch_size=50
num_epochs=4
results={}

In [3]:
#dataset
train_x= np.random.uniform(0, 2 * np.pi, 2000)
train_y= np.sin(train_x)

test_x1=np.random.uniform(-np.pi, 0, 200)
test_x2=np.random.uniform(2* np.pi, 3*np.pi, 200)
test_x=np.concatenate([test_x1, test_x2])
test_y=np.sin(test_x)

#to tensor
train_x=torch.tensor(train_x, dtype= torch.float32).view(-1,1)
train_y=torch.tensor(train_y, dtype= torch.float32).view(-1,1)
test_x=torch.tensor(test_x, dtype= torch.float32).view(-1,1)
test_y=torch.tensor(test_y, dtype= torch.float32).view(-1,1)

train_dataset = TensorDataset(train_x, train_y)
test_dataset= TensorDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [4]:
activation_functions={
    'ReLU':nn.ReLU(),
    'LeakyReLU': nn.LeakyReLU(),
    'ELU': nn.ELU(),
    'GeLU':nn.GELU(),
    'SiLU': nn.SiLU(),
}

In [5]:
class NeuralNet(nn.Module):
  def __init__(self, activation):
    super(NeuralNet, self).__init__()
    self.l1=nn.Linear(1,16)
    self.activation=activation
    self.l2=nn.Linear(16,1)
  def forward(self,x):
    x=self.l1(x)
    x=self.activation(x)
    x=self.l2(x)
    return x


In [6]:
#training loop for diff activation functions
for n, activation in activation_functions.items():
  print(f'Activation function is: {n}')

  model= NeuralNet(activation)
  optimizer=torch.optim.Adam(model.parameters(),learning_rate)
  criterion= nn.MSELoss()
  train_losses = []
  test_losses = []
  for i in range(num_epochs):
    epoch_train_loss = 0
    for x,y in train_loader:

      optimizer.zero_grad()
      output = model(x)
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
      epoch_train_loss += loss.item()

      train_losses.append(epoch_train_loss / len(train_loader))

    print(f'Epoch: {i+1}/{num_epochs} Loss: {loss.item():.3f}')

    with torch.no_grad():
        test_preds = model(test_x)
        test_loss = criterion(test_preds, test_y).item()
        test_losses.append(test_loss)

    print(f"Test Loss with {n}: {test_loss:.4f}")



Activation function is: ReLU
Epoch: 1/4 Loss: 0.968
Test Loss with ReLU: 2.8210
Epoch: 2/4 Loss: 0.499
Test Loss with ReLU: 1.8506
Epoch: 3/4 Loss: 0.443
Test Loss with ReLU: 1.6572
Epoch: 4/4 Loss: 0.310
Test Loss with ReLU: 1.7342
Activation function is: LeakyReLU
Epoch: 1/4 Loss: 0.707
Test Loss with LeakyReLU: 0.5673
Epoch: 2/4 Loss: 0.530
Test Loss with LeakyReLU: 1.0786
Epoch: 3/4 Loss: 0.350
Test Loss with LeakyReLU: 1.5600
Epoch: 4/4 Loss: 0.373
Test Loss with LeakyReLU: 1.9268
Activation function is: ELU
Epoch: 1/4 Loss: 0.204
Test Loss with ELU: 2.0859
Epoch: 2/4 Loss: 0.169
Test Loss with ELU: 2.4657
Epoch: 3/4 Loss: 0.203
Test Loss with ELU: 2.6733
Epoch: 4/4 Loss: 0.204
Test Loss with ELU: 2.7804
Activation function is: GeLU
Epoch: 1/4 Loss: 0.326
Test Loss with GeLU: 3.0022
Epoch: 2/4 Loss: 0.222
Test Loss with GeLU: 1.9058
Epoch: 3/4 Loss: 0.207
Test Loss with GeLU: 2.0165
Epoch: 4/4 Loss: 0.182
Test Loss with GeLU: 2.1233
Activation function is: SiLU
Epoch: 1/4 Loss: 0.